In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu 

In [4]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-02-26 03:03:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.37MB/s    in 0.2s    

2023-02-26 03:03:23 (5.37 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [5]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [6]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), inferSchema=True, sep='\\t', timestampFormat="yyyy-mm-dd")

# Show DataFrame
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!|2015-01-31 00:08:00|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...| 

In [7]:
# Get the number of rows in the DataFrame.
df.count()

4864249

# Transform the Data

## Create the "review_id_table".

In [8]:
# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [9]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.

review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()
review_id_df.printSchema()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RDIJS7QYB6XNR|   18778586|B00EDBY7X8|     122952789|2015-01-31 00:08:00|
|R36ED1U38IELG8|   24769659|B00D7JFOPC|     952062646|2015-01-31 00:08:00|
| R1UE3RPRGCOLD|   44331596|B002LHA74O|     818126353|2015-01-31 00:08:00|
|R298788GS6I901|   23310293|B00ARPLCGY|     261944918|2015-01-31 00:08:00|
|  RNX4EXOBBPN5|   38745832|B00UZOPOFW|     717410439|2015-01-31 00:08:00|
|R3BPETL222LMIM|   13394189|B009B7F6CA|     873028700|2015-01-31 00:08:00|
|R3SORMPJZO3F2J|    2749569|B0101EHRSM|     723424342|2015-01-31 00:08:00|
|R2RDOJQ0WBZCF6|   41137196|B00407S11Y|     383363775|2015-01-31 00:08:00|
|R2B8VBEPB4YEZ7|     433677|B00FGPU7U2|     780517568|2015-01-31 00:08:00|
|R1CB783I7B0U52|    1297934|B0013OY0S0|     269360126|2015-01-31 00:08:00|
| R2D90RQQ3V8LH|   520062

## Create the "products" Table

In [10]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()

products_df.printSchema()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IANTUPU|Lego, Star Wars, ...|
|B002TMK962|Rock - Concert Ti...|
|B00AOTTSQY|Adult & Children ...|
|B00CH3O5S6|48 Wholesale Wood...|
|B00MF3SAF2|Teenitor 3.7V 680...|
|B004D0C5WY|ThinkMax Silver B...|
|B00NHQHJZS|LEGO Technic Arct...|
|B00NQJ2DBG|Super Bright Long...|
|B0082IGA06|Yu-Gi-Oh! - Dark ...|
|1609581520|American Girl Car...|
|B004RGYVLC|DC Comics POP Her...|
|B0053EX3D4|Inuyasha Kirara T...|
|B00OTLHNI8|Bloco Toys T-Rex ...|
|B00BXEYI8W|Kid Galaxy Robot ...|
|B00JUG2NSM|Weiss Schwarz Kil...|
|B00LVPY252|Maxx Action 30" T...|
|B00IGRHWSC|Mattel Hot Wheels...|
|B00U6FU3YM|Educational Insig...|
|B00Z7FS3Q6|Beach Ball Set, 3...|
|B000WEAGTY|Inflatable Tradit...|
+----------+--------------------+
only showing top 20 rows

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



## Create the "customers" Table

In [11]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()
customers_df.printSchema()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16989307|             1|
|   45632184|             2|
|   14703850|            13|
|   49645387|             2|
|   16343477|             1|
|   15554899|             1|
|   17067926|             1|
|   50843047|             2|
|    4051424|             1|
|   11487525|             1|
|   19371753|             1|
|   18634862|             1|
|   14552054|             1|
|   52695798|             1|
|   49438424|             3|
|   10854449|             9|
|   48521319|             1|
|   11839424|             2|
|   27887950|             1|
|   45392827|             3|
+-----------+--------------+
only showing top 20 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



## Create the "vine_table".

In [12]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])
vine_df.show()
vine_df.printSchema()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RDIJS7QYB6XNR|          5|            0|          0|   N|
|R36ED1U38IELG8|          5|            0|          0|   N|
| R1UE3RPRGCOLD|          2|            1|          1|   N|
|R298788GS6I901|          5|            0|          0|   N|
|  RNX4EXOBBPN5|          1|            1|          1|   N|
|R3BPETL222LMIM|          5|            0|          0|   N|
|R3SORMPJZO3F2J|          3|            2|          2|   N|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|
|R1CB783I7B0U52|          1|            0|          1|   N|
| R2D90RQQ3V8LH|          5|            0|          0|   N|
|R1Y4ZOUGFMJ327|          5|            0|          0|   N|
|R2BUV9QJI2A00X|          5|            0|          1|   N|
| RSUHRJFJIRB3Z|          4|            

# Load

In [14]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my-db"
config = {"user":"postgres", "password": "password321", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)